In [2]:
!pip3 install pandas numpy matplotlib seaborn scikit-learn imbalanced-learn lightgbm catboost xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 MB 2.7 MB/s  0:00:44 eta 0:00:010:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.8/296.8 MB 2.5 MB/s  0:02:00 eta 0:00:010:00:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [xgboost]━━━ 1/2 [xgboost]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [4]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

In [6]:
df = pd.read_csv("./data/chicago_crime_data_processed.csv")
print(f"Dataset cargado: {df.shape}")

Dataset cargado: (350000, 23)


In [7]:
# -------------------------------------------------
# 2. TARGET Y SPLIT
# -------------------------------------------------
X = df.drop(columns=['arrest', 'date'])  # date ya no la necesitamos
y = df['arrest'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Distribución arrest en train: {y_train.mean():.2%}")

Distribución arrest en train: 6.33%


In [8]:
# -------------------------------------------------
# 3. IDENTIFICAR COLUMNAS
# -------------------------------------------------
cat_cols = X_train.select_dtypes(include=['object', 'bool']).columns.tolist()
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Quitar 'arrest' y 'date' si se colaron
cat_cols = [c for c in cat_cols if c not in ['arrest']]
num_cols = [c for c in num_cols if c not in ['arrest']]

print(f"Categóricas: {len(cat_cols)} | Numéricas: {len(num_cols)}")

Categóricas: 5 | Numéricas: 16


### 1. Catboost

In [9]:
cat_model = CatBoostClassifier(
    iterations=1200,
    learning_rate=0.05,
    depth=8,
    auto_class_weights='Balanced',      # ← balanceo automático brutal
    cat_features=cat_cols,              # ← ¡aquí la magia!
    verbose=100,
    random_state=42
)

cat_model.fit(X_train, y_train)
y_pred_cat = cat_model.predict(X_test)
y_proba_cat = cat_model.predict_proba(X_test)[:, 1]

0:	learn: 0.6705934	total: 188ms	remaining: 3m 45s
100:	learn: 0.4623836	total: 7.31s	remaining: 1m 19s
200:	learn: 0.4478130	total: 14.6s	remaining: 1m 12s
300:	learn: 0.4322628	total: 22.5s	remaining: 1m 7s
400:	learn: 0.4172989	total: 33.2s	remaining: 1m 6s
500:	learn: 0.4054030	total: 43.8s	remaining: 1m 1s
600:	learn: 0.3964632	total: 54s	remaining: 53.8s
700:	learn: 0.3857886	total: 1m 4s	remaining: 46.1s
800:	learn: 0.3756707	total: 1m 16s	remaining: 37.9s
900:	learn: 0.3662606	total: 1m 27s	remaining: 28.9s
1000:	learn: 0.3581883	total: 1m 38s	remaining: 19.5s
1100:	learn: 0.3504041	total: 1m 49s	remaining: 9.82s
1199:	learn: 0.3429188	total: 2m	remaining: 0us


In [11]:
resultados = []
auc_cat = roc_auc_score(y_test, y_proba_cat)
resultados.append({'Modelo': 'CatBoost', 'ROC-AUC': auc_cat})
print(f"CatBoost → ROC-AUC: {auc_cat:.4f}")

CatBoost → ROC-AUC: 0.8556
